# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

In [2]:
MovieLens_df = pd.read_table("ratings.dat", sep='::', header=None, names=['UserID','MovieID','Rating','Timestamp'])
MovieLens_df.drop(['Timestamp'], axis=1, inplace=True)
MovieLens_pivot = MovieLens_df.pivot_table(index='UserID', columns='MovieID', values='Rating')

In [3]:
MovieLens_pivot.shape

(6040, 3706)

In [4]:
row_index = MovieLens_pivot.isnull().sum(axis = 1).sort_values() # NaN이 적은 순으로 행 뽑아내기

In [5]:
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, index=row_index.index)

In [6]:
# rating이 많은 사용자 1,000명
MovieLens_pivot = MovieLens_pivot.iloc[:1000,:]
MovieLens_pivot.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
4169,NaN,3.0,2.0,3.0,2.0,4.0,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,1.0,4.0,4.0,5.0,3.0,NaN,4.0
1680,4.0,4.0,3.0,NaN,3.0,1.0,3.0,NaN,NaN,3.0,...,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN
4277,5.0,4.0,NaN,NaN,NaN,5.0,NaN,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,4.0,4.0,5.0,4.0,NaN,5.0
1941,5.0,4.0,3.0,NaN,3.0,NaN,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,3.0
1181,3.0,1.0,1.0,NaN,NaN,3.0,2.0,NaN,4.0,4.0,...,4.0,NaN,NaN,3.0,3.0,3.0,4.0,NaN,NaN,NaN


In [7]:
col_index = MovieLens_pivot.isnull().sum(axis = 0).sort_values() # NaN이 적은 순으로 열 뽑아내기

In [8]:
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, columns=col_index.index)

In [9]:
# rating이 많은 영화 1,000개
MovieLens_pivot = MovieLens_pivot.iloc[:,:1000]
MovieLens_pivot.head()

MovieID,1196,260,1270,1580,1210,2571,589,480,2858,608,...,1286,2457,481,2524,237,2796,2408,60,3159,538
UserID,,,,,,,,,,,,,,,,,,,,,
4169,5.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,5.0,...,2.0,4.0,2.0,3.0,3.0,3.0,3.0,NaN,NaN,4.0
1680,5.0,2.0,3.0,4.0,3.0,3.0,5.0,5.0,5.0,5.0,...,4.0,3.0,4.0,5.0,2.0,4.0,3.0,1.0,NaN,5.0
4277,5.0,5.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,...,5.0,3.0,4.0,4.0,NaN,NaN,3.0,4.0,NaN,5.0
1941,5.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,1.0,4.0,...,3.0,1.0,NaN,4.0,NaN,2.0,4.0,3.0,3.0,NaN
1181,4.0,4.0,3.0,4.0,3.0,5.0,5.0,2.0,3.0,3.0,...,2.0,2.0,3.0,2.0,NaN,NaN,NaN,2.0,NaN,3.0


In [10]:
MovieLens_pivot.shape

(1000, 1000)

In [11]:
user_index = MovieLens_pivot.index.values # User ID 기억하기
user_index[:5]

array([4169, 1680, 4277, 1941, 1181])

In [12]:
user_index = pd.Series(user_index) # Index rename 인자로 넣기 위해 형태 변경

In [13]:
MovieLens_pivot.fillna(0, inplace=True)
MovieLens_pivot = MovieLens_pivot.astype('int64')

In [14]:
MovieLens_pivot.head()

MovieID,1196,260,1270,1580,1210,2571,589,480,2858,608,...,1286,2457,481,2524,237,2796,2408,60,3159,538
UserID,,,,,,,,,,,,,,,,,,,,,
4169,5,5,4,4,5,4,4,5,5,5,...,2,4,2,3,3,3,3,0,0,4
1680,5,2,3,4,3,3,5,5,5,5,...,4,3,4,5,2,4,3,1,0,5
4277,5,5,5,4,4,5,5,5,5,5,...,5,3,4,4,0,0,3,4,0,5
1941,5,5,5,5,5,3,3,5,1,4,...,3,1,0,4,0,2,4,3,3,0
1181,4,4,3,4,3,5,5,2,3,3,...,2,2,3,2,0,0,0,2,0,3


In [15]:
MovieLens_pivot = np.array(MovieLens_pivot) # 함수 인자로 넣기 위해 numpy array 형태로 변경

# 1. Cosine Simiarity

In [16]:
def COS(data):
    size = np.size(data, axis = 0)
    simCOS = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size): # 각 유저별로 for문 반복
        for j in range(i, size):
            normI = np.linalg.norm(data[i, ]) # i벡터의 크기 계산
            normJ = np.linalg.norm(data[j, ]) # j벡터의 크기 계산
            inputData = np.dot(data[i, ], data[j, ])/(normI * normJ) # Cosine similarity 공식
            simCOS[i, j] = inputData # 행렬에 계산 값 대입하기
            simCOS[j, i] = inputData # 대각선 값 대입하기
    
    return simCOS # 최종 행렬값 반환

In [17]:
COS_result = pd.DataFrame(COS(MovieLens_pivot)) # 결과 행렬을 보기 쉽게 데이터프레임으로 출력
COS_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.787909,0.859201,0.790026,0.834528,0.791628,0.817873,0.681690,0.738135,0.818323,...,0.441598,0.470168,0.382352,0.512984,0.461470,0.490822,0.438353,0.443228,0.398683,0.510854
1,0.787909,1.000000,0.781235,0.734088,0.751072,0.695493,0.715526,0.719346,0.662814,0.751650,...,0.463808,0.409254,0.352518,0.465006,0.403417,0.463680,0.403850,0.379987,0.417151,0.421071
2,0.859201,0.781235,1.000000,0.755760,0.831464,0.751138,0.811420,0.710769,0.707323,0.791259,...,0.491438,0.454240,0.314692,0.469834,0.460844,0.517595,0.449344,0.461814,0.329651,0.488816
3,0.790026,0.734088,0.755760,1.000000,0.749956,0.669821,0.720516,0.676923,0.571850,0.698003,...,0.417122,0.491072,0.266703,0.438551,0.336850,0.431093,0.477784,0.349426,0.404217,0.395560
4,0.834528,0.751072,0.831464,0.749956,1.000000,0.764056,0.775519,0.685504,0.717929,0.761493,...,0.470280,0.463234,0.328674,0.480950,0.445125,0.499476,0.441522,0.397772,0.391696,0.508112


In [18]:
COS_result.rename(index=user_index, columns=user_index, inplace=True) # User ID mapping
COS_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.787909,0.859201,0.790026,0.834528,0.791628,0.817873,0.681690,0.738135,0.818323,...,0.441598,0.470168,0.382352,0.512984,0.461470,0.490822,0.438353,0.443228,0.398683,0.510854
1680,0.787909,1.000000,0.781235,0.734088,0.751072,0.695493,0.715526,0.719346,0.662814,0.751650,...,0.463808,0.409254,0.352518,0.465006,0.403417,0.463680,0.403850,0.379987,0.417151,0.421071
4277,0.859201,0.781235,1.000000,0.755760,0.831464,0.751138,0.811420,0.710769,0.707323,0.791259,...,0.491438,0.454240,0.314692,0.469834,0.460844,0.517595,0.449344,0.461814,0.329651,0.488816
1941,0.790026,0.734088,0.755760,1.000000,0.749956,0.669821,0.720516,0.676923,0.571850,0.698003,...,0.417122,0.491072,0.266703,0.438551,0.336850,0.431093,0.477784,0.349426,0.404217,0.395560
1181,0.834528,0.751072,0.831464,0.749956,1.000000,0.764056,0.775519,0.685504,0.717929,0.761493,...,0.470280,0.463234,0.328674,0.480950,0.445125,0.499476,0.441522,0.397772,0.391696,0.508112
889,0.791628,0.695493,0.751138,0.669821,0.764056,1.000000,0.699652,0.591424,0.719887,0.775963,...,0.457913,0.416430,0.339626,0.529679,0.479563,0.520081,0.444539,0.402079,0.421762,0.565037
3618,0.817873,0.715526,0.811420,0.720516,0.775519,0.699652,1.000000,0.645526,0.646999,0.735677,...,0.420292,0.464255,0.380855,0.474907,0.440363,0.507504,0.401359,0.461549,0.313516,0.425799
2063,0.681690,0.719346,0.710769,0.676923,0.685504,0.591424,0.645526,1.000000,0.510674,0.640159,...,0.499784,0.447565,0.346715,0.425408,0.339126,0.455756,0.381324,0.394409,0.333623,0.296792
1150,0.738135,0.662814,0.707323,0.571850,0.717929,0.719887,0.646999,0.510674,1.000000,0.719577,...,0.391289,0.315031,0.337551,0.488104,0.502838,0.463840,0.315984,0.384958,0.359010,0.596668
1015,0.818323,0.751650,0.791259,0.698003,0.761493,0.775963,0.735677,0.640159,0.719577,1.000000,...,0.473319,0.456470,0.391897,0.499309,0.511887,0.525590,0.421564,0.439472,0.388707,0.523197


# 2. Pearson Correlation Coefficient

In [19]:
def cal_PCC(i, j): # 두 벡터의 PCC값을 계산해 주는 함수
    i = np.array(i) # numpy array로 형변환
    j = np.array(j) # numpy array로 형변환
    
    # 벡터의 0인 원소를 null값으로 바꾸고 null값을 제외한 평균값
    mean_i = np.nanmean(np.where(i != 0, i, np.nan)) 
    mean_j = np.nanmean(np.where(j != 0, j, np.nan))
    
    # 벡터의 원소가 0인 부분의 인덱스를 저장한 리스트 생성
    zero_i = np.where(i == 0)
    zero_j = np.where(j == 0)
    
    # zero_i와 zero_j 벡터를 하나로 합쳐준다.
    zeros = np.concatenate((zero_i, zero_j), axis = None)
    
    # 각 벡터의 원소가 0인 인덱스를 삭제
    del_i = np.delete(i, zeros) 
    del_j = np.delete(j, zeros)
    
    # 각 벡터의 원소에서 평균값을 빼준다.
    del_i = del_i - mean_i
    del_j = del_j - mean_j
    
    return np.dot(del_i, del_j)/(np.linalg.norm(del_i) * np.linalg.norm(del_j))
    # PCC 공식을 통한 similarity 값 반환

def PCC(data):
    size = np.size(data, axis = 0)
    simPCC = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            inputData = cal_PCC(data[i, ], data[j, ]) # cal_PCC함수를 이용
            simPCC[i, j] = inputData
            simPCC[j, i] = inputData
    
    return simPCC

In [20]:
PCC_result = pd.DataFrame(PCC(MovieLens_pivot))
PCC_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.211497,0.354400,0.293246,0.293276,0.372993,0.274616,0.148154,0.306337,0.415807,...,0.304526,0.277605,0.396047,0.352337,0.286388,0.127263,0.310353,0.316427,0.356361,0.333957
1,0.211497,1.000000,0.149494,0.088131,0.129940,0.220241,0.024323,0.114591,0.215764,0.265320,...,0.162364,0.012271,0.143634,0.139583,0.184275,-0.022524,0.176602,0.169504,0.142366,0.227280
2,0.354400,0.149494,1.000000,0.229283,0.290458,0.349931,0.187860,0.194284,0.199182,0.313583,...,0.221930,0.277117,0.371396,0.098076,0.231483,0.284210,0.331549,0.307576,0.359891,0.280653
3,0.293246,0.088131,0.229283,1.000000,0.178404,0.171008,0.242260,0.100666,0.063471,0.140622,...,0.140930,0.496504,0.163696,0.107726,0.080942,0.116291,0.316086,0.091718,0.340129,0.105009
4,0.293276,0.129940,0.290458,0.178404,1.000000,0.337146,0.106232,0.181197,0.219753,0.282102,...,0.220066,0.096540,0.271159,0.226355,0.199116,0.281512,0.232585,0.284388,0.263320,0.283798


In [21]:
PCC_result.rename(index=user_index, columns=user_index, inplace=True)
PCC_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.211497,0.354400,0.293246,0.293276,0.372993,0.274616,0.148154,0.306337,0.415807,...,0.304526,0.277605,0.396047,0.352337,0.286388,0.127263,0.310353,0.316427,0.356361,0.333957
1680,0.211497,1.000000,0.149494,0.088131,0.129940,0.220241,0.024323,0.114591,0.215764,0.265320,...,0.162364,0.012271,0.143634,0.139583,0.184275,-0.022524,0.176602,0.169504,0.142366,0.227280
4277,0.354400,0.149494,1.000000,0.229283,0.290458,0.349931,0.187860,0.194284,0.199182,0.313583,...,0.221930,0.277117,0.371396,0.098076,0.231483,0.284210,0.331549,0.307576,0.359891,0.280653
1941,0.293246,0.088131,0.229283,1.000000,0.178404,0.171008,0.242260,0.100666,0.063471,0.140622,...,0.140930,0.496504,0.163696,0.107726,0.080942,0.116291,0.316086,0.091718,0.340129,0.105009
1181,0.293276,0.129940,0.290458,0.178404,1.000000,0.337146,0.106232,0.181197,0.219753,0.282102,...,0.220066,0.096540,0.271159,0.226355,0.199116,0.281512,0.232585,0.284388,0.263320,0.283798
889,0.372993,0.220241,0.349931,0.171008,0.337146,1.000000,0.212324,0.120055,0.336472,0.417022,...,0.209822,0.269488,0.100604,0.255423,0.186251,0.257275,0.377171,0.284053,0.196564,0.340289
3618,0.274616,0.024323,0.187860,0.242260,0.106232,0.212324,1.000000,0.023920,0.138970,0.164975,...,0.205570,0.384482,0.092457,0.230607,0.088271,0.229450,0.317554,0.260748,0.198316,-0.009276
2063,0.148154,0.114591,0.194284,0.100666,0.181197,0.120055,0.023920,1.000000,-0.050732,0.176501,...,0.263054,0.085337,0.220300,0.195391,-0.046394,0.188205,0.020901,0.108741,0.085320,0.062183
1150,0.306337,0.215764,0.199182,0.063471,0.219753,0.336472,0.138970,-0.050732,1.000000,0.306358,...,0.361817,0.073494,0.089480,0.292553,0.262534,0.081424,0.022269,0.303639,0.407771,0.284193
1015,0.415807,0.265320,0.313583,0.140622,0.282102,0.417022,0.164975,0.176501,0.306358,1.000000,...,0.235265,0.276155,0.324072,0.291163,0.265516,0.215041,0.281610,0.380880,0.345359,0.258085


# 3. JMSD

In [22]:
def MSD(data):
    size = np.size(data, axis = 0)
    simMSD = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            # 벡터의 원소가 0인 부분을 null값으로 변경
            nozero_i = np.where(data[i, ] == 0, np.nan, data[i, ])
            nozero_j = np.where(data[j, ] == 0, np.nan, data[j, ])
            
            # i벡터에서 j벡터의 각 원소를 빼고 제곱한 결과
            SquaredSum = np.square(nozero_i - nozero_j)
            
            # 제곱한 결과가 null값인 것을 제외
            SquaredSum = SquaredSum[~np.isnan(SquaredSum)]
            
            # MSD 공식에 따른 similarity 값
            AllItems = np.size(SquaredSum, axis=0)
            tmp = np.sum(SquaredSum)/AllItems
            
            # 행렬에 구한 similarity 값 대입
            simMSD[i, j] = tmp
            simMSD[j, i] = simMSD[i, j]
            
    return simMSD

In [23]:
def JAC(data):
    size = np.size(data, axis = 0)
    simJAC = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성
    
    for i in range(0, size):
        for j in range(i, size):
            # 벡터의 원소가 0이상을 만족하는 인덱스만 1, 0이라면 0
            I = np.array(data[i, ] > 0, dtype = np.int)
            J = np.array(data[j, ] > 0, dtype = np.int)
            
            #I, J벡터의 합
            SumIJ = I + J
            
            # 합한 원소가 1보다 크면 교집합
            Inter = np.sum(np.array(SumIJ > 1, dtype=np.int))
            # 0보다 큰 모든 원소의 합은 합집합
            Union = np.sum(np.array(SumIJ > 0, dtype=np.int))
            
            # JAC 공식을 따른 similarity
            tmp = Inter / Union
            
            # 결과 값 simJAC행렬에 대입
            simJAC[i, j] = tmp
            simJAC[j, i] = simJAC[i, j]
            
    return simJAC

In [24]:
def JMSD(data, max):
    return JAC(data) * (1 - (MSD(data/max))) # JMSD 공식에 따른 similarity, max는 최대 평가 값

In [25]:
JMSD_result = pd.DataFrame(JMSD(MovieLens_pivot, 5))
JMSD_result.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.665878,0.731176,0.660820,0.701961,0.600085,0.665748,0.566173,0.513397,0.640678,...,0.214785,0.232304,0.162960,0.259730,0.190045,0.223120,0.211665,0.195709,0.182094,0.211076
1,0.665878,1.000000,0.632191,0.624619,0.622163,0.520085,0.560170,0.598473,0.460108,0.580450,...,0.244434,0.210991,0.155699,0.240331,0.187094,0.221775,0.202275,0.180851,0.195561,0.190871
2,0.731176,0.632191,1.000000,0.601782,0.656456,0.546249,0.632205,0.537364,0.462383,0.615043,...,0.245317,0.218447,0.118714,0.230950,0.198607,0.243152,0.213493,0.208637,0.148087,0.228924
3,0.660820,0.624619,0.601782,1.000000,0.641247,0.516441,0.575880,0.586870,0.436571,0.531078,...,0.212062,0.244509,0.130376,0.244010,0.155379,0.199098,0.232950,0.168217,0.198725,0.169234
4,0.701961,0.622163,0.656456,0.641247,1.000000,0.602678,0.646371,0.579979,0.545562,0.579533,...,0.216468,0.233559,0.157422,0.254042,0.200980,0.223869,0.219442,0.183080,0.175777,0.216765


In [26]:
JMSD_result.rename(index=user_index, columns=user_index, inplace=True)
JMSD_result

,4169,1680,4277,1941,1181,889,3618,2063,1150,1015,...,3158,5087,4430,1937,131,4560,516,2700,5794,2185
4169,1.000000,0.665878,0.731176,0.660820,0.701961,0.600085,0.665748,0.566173,0.513397,0.640678,...,0.214785,0.232304,0.162960,0.259730,0.190045,0.223120,0.211665,0.195709,0.182094,0.211076
1680,0.665878,1.000000,0.632191,0.624619,0.622163,0.520085,0.560170,0.598473,0.460108,0.580450,...,0.244434,0.210991,0.155699,0.240331,0.187094,0.221775,0.202275,0.180851,0.195561,0.190871
4277,0.731176,0.632191,1.000000,0.601782,0.656456,0.546249,0.632205,0.537364,0.462383,0.615043,...,0.245317,0.218447,0.118714,0.230950,0.198607,0.243152,0.213493,0.208637,0.148087,0.228924
1941,0.660820,0.624619,0.601782,1.000000,0.641247,0.516441,0.575880,0.586870,0.436571,0.531078,...,0.212062,0.244509,0.130376,0.244010,0.155379,0.199098,0.232950,0.168217,0.198725,0.169234
1181,0.701961,0.622163,0.656456,0.641247,1.000000,0.602678,0.646371,0.579979,0.545562,0.579533,...,0.216468,0.233559,0.157422,0.254042,0.200980,0.223869,0.219442,0.183080,0.175777,0.216765
889,0.600085,0.520085,0.546249,0.516441,0.602678,1.000000,0.545130,0.453390,0.515885,0.575728,...,0.239560,0.218255,0.166255,0.304572,0.224034,0.249638,0.243839,0.194645,0.185254,0.261636
3618,0.665748,0.560170,0.632205,0.575880,0.646371,0.545130,1.000000,0.517397,0.488630,0.558643,...,0.214348,0.231291,0.168624,0.254630,0.214740,0.255762,0.205907,0.231924,0.135183,0.189831
2063,0.566173,0.598473,0.537364,0.586870,0.579979,0.453390,0.517397,1.000000,0.402142,0.485603,...,0.251353,0.251375,0.166931,0.227548,0.161228,0.222310,0.202075,0.189974,0.174188,0.136168
1150,0.513397,0.460108,0.462383,0.436571,0.545562,0.515885,0.488630,0.402142,1.000000,0.486226,...,0.194986,0.178936,0.173769,0.280353,0.231843,0.232024,0.170210,0.196578,0.162783,0.263467
1015,0.640678,0.580450,0.615043,0.531078,0.579533,0.575728,0.558643,0.485603,0.486226,1.000000,...,0.251168,0.245469,0.178290,0.256243,0.235851,0.257839,0.214096,0.205462,0.180980,0.249684
